In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, RSI 
from quantopian.pipeline import CustomFilter
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
start = '2015-6-1'
end = '2016-7-1'

In [ ]:
class SidInList(CustomFilter):  
    """  
    Filter returns True for any SID included in parameter tuple passed at creation.  
    Usage: my_filter = SidInList(sid_list=(23911, 46631))  
    """  
    inputs = []  
    window_length = 1  
    params = ('sid_list',)

    def compute(self, today, assets, out, sid_list):  
        out[:] = np.in1d(assets, sid_list)

In [ ]:
def make_pipeline():

    s = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=35)
    l = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=50)
    rsi = RSI(inputs=[USEquityPricing.close], window_length=10)
    close = USEquityPricing.close.latest
    symbol = SidInList(sid_list=(8554))
    
    return Pipeline(
        columns={
            'short':s,
            'long': l,
            'rsi':rsi,
            'close': close,
            'symbol': symbol,
       },
       screen=symbol,
    )

In [ ]:
result = run_pipeline(make_pipeline(), start, end)

In [ ]:
rsi = result.rsi.values
shorts = result.short.values 
longs = result.long.values
price = result.close.values
dates = [x[0].to_datetime() for x in result.index.tolist()]

plt.plot(dates, shorts, color='r')
plt.plot(dates, longs, color='b')
plt.plot(dates, price, color='c')


In [ ]:
plt.plot(dates, rsi)
plt.plot(dates, [55]*len(dates))
plt.plot(dates, [45]*len(dates))